In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import pandas as pd
from pathlib import Path

# butterworth filter the data to clean the spikes
from scipy.signal import butter, filtfilt

try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets
    import ipywidgets

In [ ]:
# Experiment parameters and thresholds
frame_rate = 300 # Hz
velocity_threshold = -2000 # mm/s
acceleration_threshold = 0.1 # mm/s^2
jerk_threshold = 0 # mm/s^3

### Threshold guidelines?
- possibly use relative threshold based on body weight? 
- perhaps an absolute threshold of velocity zero crossing is good enough 
- visual inspection? Kinda tedious. 

In [ ]:
pickle_path = Path(r"C:\Users\miken\DATA\ARGP\SAMPLE_ARGP_DATA\2022-08-29_Pilot_Data0002\generic_skelly_dict.pkl")

In [ ]:
# open the saved file and load the dictionary using pickle
with open(pickle_path, 'rb') as f:
    generic_skelly_dict = pickle.load(f)


In [ ]:
# Listing out the keys in the dictionary for reference 
key_anatomical_points = list(generic_skelly_dict.keys())
print(key_anatomical_points)

In [ ]:
# Extract the heel data from the dictionary
left_heel_data = generic_skelly_dict['left_heel_xyz']
right_heel_data = generic_skelly_dict['right_heel_xyz']

# extract the x,y,z coordinates from the heel data
left_heel_x = left_heel_data[:,0]
left_heel_y = left_heel_data[:,1]
left_heel_z = left_heel_data[:,2]

right_heel_x = right_heel_data[:,0]
right_heel_y = right_heel_data[:,1]
right_heel_z = right_heel_data[:,2]

In [ ]:
print(len(left_heel_data))

In [ ]:
# Create a time vector
def create_time_vector(left_heel_data, frame_rate): # Could have used any key in the dictionary here
    time_vector = np.arange(0, len(left_heel_data)/frame_rate, 1/frame_rate)
    return time_vector

time_vector = create_time_vector(left_heel_data, frame_rate)

print(len(time_vector))

In [ ]:
plt.plot(left_heel_data)
plt.show()

In [ ]:
# filter the data to clean the spikes in derived data
def butterworth_filter(data, cutoff, frame_rate, order=4, filter_type='low'):
    nyq = 0.5 * frame_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)
    
    # Adjust the padlen based on the length of the data
    padlen = min(order * 3, len(data) - 1)
    
    y = filtfilt(b, a, data, padlen=padlen)  
    return y 


In [ ]:
# Filter parameters
cutoff_frequency = 10 # Hz
frame_rate = 300 # Hz
filter_order = 4 # Filter order

filtered_left_heel_z = butterworth_filter(left_heel_z, cutoff_frequency, frame_rate, order=filter_order)
filtered_right_heel_z = butterworth_filter(right_heel_z, cutoff_frequency, frame_rate, order=filter_order)

In [ ]:
# Calculate the velocity and acceleration from position data
def calculate_velocity_acceleration_jerk(position_data, frame_rate):
    velocity_data = np.diff(position_data) * frame_rate
    acceleration_data = np.diff(velocity_data) * frame_rate
    jerk_data = np.diff(acceleration_data) * frame_rate
    return velocity_data, acceleration_data, jerk_data

# Find heel strikes based on velocity and acceleration (1D)
def find_heel_strikes_velocity_acceleration_jerk(heel_position_data, frame_rate, velocity_threshold, acceleration_threshold):
    velocity_data, acceleration_data, jerk_data = calculate_velocity_acceleration_jerk(heel_position_data, frame_rate)

    heel_strike_indices = []
    window_size = 60  # Set the window size for local minimum and maximum checks

    for i in range(window_size, len(velocity_data) - window_size):
        # Check if the current index is a local minimum for velocity data within the window
        is_local_min = np.argmin(velocity_data[i - window_size:i + window_size + 1]) == window_size

        # Check if the current index is a local maximum for jerk data within the window
        is_local_max = np.argmax(jerk_data[i - window_size:i + window_size + 1]) == window_size

        # If both conditions are true, store the index as a heel strike
        if velocity_data[i] < velocity_threshold and is_local_max:
            heel_strike_indices.append(i)

    # return heel_strike_indices
    # heel_strike_indices = []
    # for i in range(len(velocity_data)-1):  # Adjust the range to avoid index errors
    #     if velocity_data[i] < velocity_threshold and acceleration_data[i] > acceleration_threshold:
    #         heel_strike_indices.append(i+1)  # Add 1 to the index because diff causes you to lose one data point

    return heel_strike_indices

In [ ]:
# Call the function for both left and right heel position data
left_heel_strikes_indices = find_heel_strikes_velocity_acceleration_jerk(filtered_left_heel_z, frame_rate, velocity_threshold, acceleration_threshold)
right_heel_strikes_indices = find_heel_strikes_velocity_acceleration_jerk(filtered_right_heel_z, frame_rate, velocity_threshold, acceleration_threshold)

In [ ]:
# Get the heel strike times using the time vector
left_heel_strikes_times = time_vector[left_heel_strikes_indices]
right_heel_strikes_times = time_vector[right_heel_strikes_indices]

# Create a dictionary with left and right heel strikes
heel_strikes_dict = {
    'left': {
        'indices': left_heel_strikes_indices,
        'times': left_heel_strikes_times,
        'positions': left_heel_data[left_heel_strikes_indices]
    },
    'right': {
        'indices': right_heel_strikes_indices,
        'times': right_heel_strikes_times,
        'positions': right_heel_data[right_heel_strikes_indices]
    }
}



In [ ]:
print(heel_strikes_dict['right']['times'])

In [ ]:
print(len(heel_strikes_dict['right']['times']))

In [ ]:
r_heel_strike = list(heel_strikes_dict['right'].keys())
print(r_heel_strike)

### I want to check if the "positions" actually hold the correct values compared to the debug plot below

In [ ]:
# debug plot to see if anything is actually showing 

left_heel_z_velocity, left_heel_z_acceleration, left_heel_z_jerk = calculate_velocity_acceleration_jerk(filtered_left_heel_z, frame_rate)


# Plot the left heel's z-axis velocity and acceleration data
plt.figure(figsize=(12, 6))

time_vector_zv = time_vector[:-1] # Adjust the time vector to match the velocity and acceleration data
time_vector_za = time_vector[:-2] 
time_vector_zj = time_vector[:-3]

plt.subplot(3, 1, 1)
plt.plot(time_vector_zv, left_heel_z_velocity, label='Left heel Z velocity', color='blue')
#plt.plot(time_vector_zv, right_heel_data_z_velocity, label='Right heel Z velocity', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Velocity')
plt.legend()
plt.xlim(40,45)

plt.subplot(3, 1, 2)
plt.plot(time_vector_za, left_heel_z_acceleration, label='Left heel Z acceleration')
plt.xlabel('Time (s)')
plt.ylabel('Acceleration')
plt.legend()
plt.xlim(40,45)

plt.subplot(3, 1, 3)
plt.plot(time_vector_zj, left_heel_z_jerk, label='Left heel Z jerk')
plt.xlabel('Time (s)')
plt.ylabel('Jerk')
plt.legend()
plt.xlim(40,45)

plt.show()

In [ ]:
# Find the indices corresponding to 40 seconds and 42 seconds
start_time = 40
end_time = 45
start_index = np.argmin(np.abs(time_vector - start_time))
end_index = np.argmin(np.abs(time_vector - end_time))

# Get the data in the dictionary for the specified time range
filtered_heel_strikes_dict = {
    'left': {
        'indices': [idx for idx in heel_strikes_dict['left']['indices'] if start_index <= idx <= end_index],
        'times': heel_strikes_dict['left']['times'][(heel_strikes_dict['left']['times'] >= start_time) & (heel_strikes_dict['left']['times'] <= end_time)],
        'positions': heel_strikes_dict['left']['positions'][(heel_strikes_dict['left']['times'] >= start_time) & (heel_strikes_dict['left']['times'] <= end_time)]
    },
    'right': {
        'indices': [idx for idx in heel_strikes_dict['right']['indices'] if start_index <= idx <= end_index],
        'times': heel_strikes_dict['right']['times'][(heel_strikes_dict['right']['times'] >= start_time) & (heel_strikes_dict['right']['times'] <= end_time)]
    }
}


print(len(filtered_heel_strikes_dict['left']['indices']))